In [1]:
from astropy.time import Time, TimeDelta
import astropy.units as u
import glob
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sunpy.coordinates.sun as sn
import scipy.ndimage as ndi
import scipy.stats as st
# Our own library for using spice with STEREO (https://github.com/LukeBarnard/stereo_spice)
from stereo_spice.coordinates import StereoSpice
# Local packages
import HUXt as H

spice = StereoSpice()


In [4]:
def setup_huxt(start_time, wind='uniform'):
    """
    Initialise HUXt with some predetermined boundary/initial conditions
    start_time should be astropy.Time object.
    wind should be uniform or structured
    """
    cr_num = np.fix(sn.carrington_rotation_number(start_time))
    ert = H.Observer('EARTH', start_time)

    # Set up HUXt for a 5 day simulation with homogenous inner boundary.
    vr_in, br_in = H.Hin.get_MAS_long_profile(cr_num, ert.lat.to(u.deg))
    if wind == 'uniform':
        vr_in = np.zeros(vr_in.shape) + 400*vr_in.unit
        
    model = H.HUXt(v_boundary=vr_in, cr_num=cr_num, cr_lon_init=ert.lon_c, latitude=ert.lat.to(u.deg),
                   br_boundary=br_in, lon_start=270*u.deg, lon_stop=90*u.deg, simtime=3.5*u.day, dt_scale=4)
    
    return model

Files already exist for CR2089


t:   0%|                                                                             | 0/168 [00:00<?, ?it/s, now=None]

Moviepy - Building video C:\Users\yq904481\research\repos\SIR_HUXt\figures\HUXt_figures\HUXt_CR2089_uniform_movie.mp4.
Moviepy - Writing video C:\Users\yq904481\research\repos\SIR_HUXt\figures\HUXt_figures\HUXt_CR2089_uniform_movie.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\yq904481\research\repos\SIR_HUXt\figures\HUXt_figures\HUXt_CR2089_uniform_movie.mp4
Files already exist for CR2089


t:   0%|                                                                             | 0/168 [00:00<?, ?it/s, now=None]

Moviepy - Building video C:\Users\yq904481\research\repos\SIR_HUXt\figures\HUXt_figures\HUXt_CR2089_structured_movie.mp4.
Moviepy - Writing video C:\Users\yq904481\research\repos\SIR_HUXt\figures\HUXt_figures\HUXt_CR2089_structured_movie.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\yq904481\research\repos\SIR_HUXt\figures\HUXt_figures\HUXt_CR2089_structured_movie.mp4
